# Azure Kusto
In this notebook we will access Azure Kusto database

In [2]:
# install all the azure kusto modules
%pip install azure.kusto.data
%pip install azure-kusto-ingest
%pip install azure.identity
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [15]:
from azure.kusto.data import KustoClient, KustoConnectionStringBuilder, ClientRequestProperties
from azure.identity import AzureCliCredential, ChainedTokenCredential, DefaultAzureCredential, ManagedIdentityCredential
from azure.kusto.data.exceptions import KustoServiceError
from azure.kusto.data.helpers import dataframe_from_result_table
import pandas as pd
import json

class KustoConnection():
    def __init__(self, configuration : dict):
        uri = configuration.get('cluster_uri')
        if (not str(uri).startswith('https://')):
            uri = "https://" + uri
        if (not str(uri).endswith('.kusto.windows.net')) :
            uri = uri + '.kusto.windows.net'
        self.cluster_uri = uri
        self.database = configuration.get('database')
        cred = ChainedTokenCredential(ManagedIdentityCredential(), AzureCliCredential(), DefaultAzureCredential())
        connection_string = KustoConnectionStringBuilder.with_aad_application_token_authentication(self.cluster_uri, 
                                                                                                   cred.get_token(self.cluster_uri + '/.default').token)        
        # Create a KustoClient instance
        self.client = KustoClient(connection_string)

    def query(self, query: str, properties: ClientRequestProperties = None) -> pd.DataFrame:
        """
        Execute a Kusto query.
        @param query: The query.
        @returns: A Pandas DataFrame.
        """
        response = self.client.execute_query(self.database, query, properties)
        df = dataframe_from_result_table(response.primary_results[0])
        return df

    def get_schema(self, table_name: str) -> dict:
        """
        Get a table schema
        @param table_name: name of table
        @returns a dictionary mapping column names to kusto types
        """
        schema = self.client.execute_mgmt(self.database, f".show table {table_name} schema as json").primary_results[0][0]['Schema']
        return json.loads(schema)

    def close(self):
        """
        Close the Kusto client.
        """
        self.client.close()
    
cluster_uri = "dpxedwhprod.southcentralus"
database_name = "Dpxedwhprod"

kusto_client = KustoConnection({'cluster_uri' : cluster_uri, 'database':database_name})
# Query the database
query = "DCEPAllMilestonesCurrent() | take 100"
response = kusto_client.query(query)
print(response)

response = kusto_client.get_schema('FacilityMasterDimView')
print(response)

kusto_client.close()



                  PORVersion SnapshotVersion  ColoTrancheId      Region  \
0  2023-06-30 00:00:00+00:00          Latest          20248  AMERS WEST   
1  2023-06-30 00:00:00+00:00          Latest          20249  AMERS WEST   
2  2023-06-30 00:00:00+00:00          Latest          20255  AMERS WEST   
3  2023-06-30 00:00:00+00:00          Latest          20261  AMERS WEST   
4  2023-06-30 00:00:00+00:00          Latest          20260  AMERS WEST   
..                       ...             ...            ...         ...   
95 2023-06-30 00:00:00+00:00          Latest          19437        EMEA   
96 2023-06-30 00:00:00+00:00          Latest          19341        EMEA   
97 2023-06-30 00:00:00+00:00          Latest          19578        EMEA   
98 2023-06-30 00:00:00+00:00          Latest          19576        EMEA   
99 2023-06-30 00:00:00+00:00          Latest          20161        EMEA   

        SubRegion        Country                     Campus         Metro  \
0   United States  Uni